In [1]:
from Bio import SeqIO
import glob
import re

In [2]:
#create list of reference genbank files
gb_files = glob.glob("data/genbank/*.gb")
gb_files

['data/genbank/Enterobacteria_phage_UAB_Phi87.gb',
 'data/genbank/Enterobacteria_phage_WV8_complete_genome.gb',
 'data/genbank/Escherichia_coli_O157_typing_phage_1_complete_genome.gb',
 'data/genbank/Escherichia_phage vB_EcoM_ASO1A_complete_genome.gb',
 'data/genbank/Escherichia_phage_EC6_complete_genome.gb',
 'data/genbank/Escherichia_phage_HY02_complete_genome.gb',
 'data/genbank/Escherichia_phage_JH2_complete_genome.gb',
 'data/genbank/Escherichia_phage_vB_EcoM_Alf5_complete_genome.gb',
 'data/genbank/Escherichia_phage_vB_EcoM_ASO1B_complete_genome.gb',
 'data/genbank/Escherichia_phage_vB_EcoM_AYO145A_complete_genome.gb',
 'data/genbank/Escherichia_phage_vB_EcoM_VpaE1_complete_genome.gb',
 'data/genbank/Salmonella_phage_BPS15Q2_complete_genome.gb',
 'data/genbank/Salmonella_phage_BPS17L1_complete_genome.gb',
 'data/genbank/Salmonella_phage_BPS17W1_complete_genome.gb',
 'data/genbank/Salmonella_phage_Mushroom_complete_genome.gb',
 'data/genbank/Salmonella_phage_vB_SPuM_SP116_complete

In [3]:
## grab coding regions from genbank - and translate
for file in gb_files:
    infile= file
    outfile = 'data/faa/' + infile.split('/')[-1].split('.')[0] + '.faa'
    input_handle  = open(infile, "r")
    output_handle = open(outfile, "w")
    file_name = infile.split('.')[0]
    
    for seq_record in SeqIO.parse(input_handle, "genbank") :
        print("Dealing with GenBank record %s" % file_name)
        output_handle.write('>' + file_name + '\n')
        for seq_feature in seq_record.features :
            if seq_feature.type=="CDS" :
                assert len(seq_feature.qualifiers['translation'])==1
                output_handle.write(
                   seq_feature.qualifiers['translation'][0])
        output_handle.write('\n')

    output_handle.close()
    input_handle.close()

Dealing with GenBank record data/genbank/Enterobacteria_phage_UAB_Phi87
Dealing with GenBank record data/genbank/Enterobacteria_phage_WV8_complete_genome
Dealing with GenBank record data/genbank/Escherichia_coli_O157_typing_phage_1_complete_genome
Dealing with GenBank record data/genbank/Escherichia_phage vB_EcoM_ASO1A_complete_genome
Dealing with GenBank record data/genbank/Escherichia_phage_EC6_complete_genome
Dealing with GenBank record data/genbank/Escherichia_phage_HY02_complete_genome
Dealing with GenBank record data/genbank/Escherichia_phage_JH2_complete_genome
Dealing with GenBank record data/genbank/Escherichia_phage_vB_EcoM_Alf5_complete_genome
Dealing with GenBank record data/genbank/Escherichia_phage_vB_EcoM_ASO1B_complete_genome
Dealing with GenBank record data/genbank/Escherichia_phage_vB_EcoM_AYO145A_complete_genome
Dealing with GenBank record data/genbank/Escherichia_phage_vB_EcoM_VpaE1_complete_genome
Dealing with GenBank record data/genbank/Salmonella_phage_BPS15Q2_co

In [5]:
##combine faa files into one
!cat data/faa/*.faa > data/aa_seqs.fasta

In [7]:
##MSA
from Bio.Align.Applications import ClustalOmegaCommandline
in_file = "data/aa_seqs.fasta"
out_file = "data/aa_seqs_omega_aln.fasta"
clustalomega_cline = ClustalOmegaCommandline(infile=in_file, outfile=out_file, verbose=True, auto=True, 
                                            distmat_out=True, percentid=True, distmat_full=True)
print(clustalomega_cline)

clustalo -i data/aa_seqs.fasta --distmat-out True --full --percent-id -o data/aa_seqs_omega_aln.fasta --auto -v


In [ ]:
# #Run and Time MSA algorithm
# import time

# start_time = time.time()

# clustalomega_cline()

# end_time = time.time()
# execution_time = end_time - start_time
# print("Execution time:", execution_time, "seconds")

In [8]:
!clustalo -i data/aa_seqs.fasta --distmat-out True --full --percent-id -o data/aa_seqs_omega_aln.fasta --auto -v

Using 16 threads
Read 16 sequences (type: Protein) from data/aa_seqs.fasta
not more sequences (16) than cluster-size (100), turn off mBed
Setting options automatically based on input sequence characteristics (might overwrite some of your options).
Auto settings: Enabling mBed.
Auto settings: Setting iteration to 1.
Using 15 seeds (chosen with constant stride from length sorted seqs) for mBed (from a total of 16 sequences)
Calculating pairwise ktuple-distances...
Ktuple-distance calculation progress done. CPU time: 100.49u 0.07s 00:01:40.55 Elapsed: 00:00:47
mBed created 1 cluster/s (with a minimum of 1 and a soft maximum of 100 sequences each)
Distance calculation within sub-clusters done. CPU time: 96.94u 0.02s 00:01:36.96 Elapsed: 00:00:45
Guide-tree computation (mBed) done.
Ignoring request to write distance matrix (am in mBed mode)
Progressive alignment progress done. CPU time: 754.90u 32.63s 00:13:07.53 Elapsed: 00:08:54
Iteration step 1 out of 1
Computing new guide tree (iteratio

In [9]:
##double check fasta formatting for alignment
SeqIO.parse('data/aa_seqs_omega_aln.fasta', "fasta")

In [10]:
##check if the number of nucleotide are equal
!awk '/^>/{if (l!="") print l; print; l=0; next}{l+=length($0)}END{print l}' data/aa_seqs_omega_aln.fasta |paste - -

>data/genbank/Enterobacteria_phage_UAB_Phi87	45694
>data/genbank/Enterobacteria_phage_WV8_complete_genome	45694
>data/genbank/Escherichia_coli_O157_typing_phage_1_complete_genome	45694
>data/genbank/Escherichia_phage_EC6_complete_genome	45694
>data/genbank/Escherichia_phage_HY02_complete_genome	45694
>data/genbank/Escherichia_phage_JH2_complete_genome	45694
>data/genbank/Escherichia_phage_vB_EcoM_Alf5_complete_genome	45694
>data/genbank/Escherichia_phage vB_EcoM_ASO1A_complete_genome	45694
>data/genbank/Escherichia_phage_vB_EcoM_ASO1B_complete_genome	45694
>data/genbank/Escherichia_phage_vB_EcoM_AYO145A_complete_genome	45694
>data/genbank/Escherichia_phage_vB_EcoM_VpaE1_complete_genome	45694
>data/genbank/Salmonella_phage_BPS15Q2_complete_genome	45694
>data/genbank/Salmonella_phage_BPS17L1_complete_genome	45694
>data/genbank/Salmonella_phage_BPS17W1_complete_genome	45694
>data/genbank/Salmonella_phage_Mushroom_complete_genome	45694
>data/genbank/Salmonella_phage_vB_SPuM_SP116_complete_

## Running Command line Tool - RaXml 

In [11]:
!./bin/modeltest-ng --help

                             _      _ _            _      _   _  _____ 
                            | |    | | |          | |    | \ | |/ ____|
         _ __ ___   ___   __| | ___| | |_ ___  ___| |_   |  \| | |  __ 
        | '_ ` _ \ / _ \ / _` |/ _ \ | __/ _ \/ __| __|  | . ` | | |_ |
        | | | | | | (_) | (_| |  __/ | ||  __/\__ \ |_   | |\  | |__| |
        |_| |_| |_|\___/ \__,_|\___|_|\__\___||___/\__|  |_| \_|\_____|
--------------------------------------------------------------------------------
ModelTest-NG v0.1.7 released on 17.03.2021 by The Exelixis Lab.
Written by Diego Darriba.
Contributors: Tomas Flouri, Alexey Kozlov, Benoit Morel, David Posada, 
              Alexandros Stamatakis.
Latest version: https://github.com/ddarriba/modeltest
--------------------------------------------------------------------------------

Usage: modeltest-ng -i sequenceFilename
            [-c n_categories] [-d nt|aa] [-F] [-N]
            [-p numberOfThreads] [-q partitionsFile]
        

In [12]:
!./bin/modeltest-ng -i data/aa_seqs_omega_aln.fasta -d aa -p 16

                             _      _ _            _      _   _  _____ 
                            | |    | | |          | |    | \ | |/ ____|
         _ __ ___   ___   __| | ___| | |_ ___  ___| |_   |  \| | |  __ 
        | '_ ` _ \ / _ \ / _` |/ _ \ | __/ _ \/ __| __|  | . ` | | |_ |
        | | | | | | (_) | (_| |  __/ | ||  __/\__ \ |_   | |\  | |__| |
        |_| |_| |_|\___/ \__,_|\___|_|\__\___||___/\__|  |_| \_|\_____|
--------------------------------------------------------------------------------
ModelTest-NG v0.1.7 released on 17.03.2021 by The Exelixis Lab.
Written by Diego Darriba.
Contributors: Tomas Flouri, Alexey Kozlov, Benoit Morel, David Posada, 
              Alexandros Stamatakis.
Latest version: https://github.com/ddarriba/modeltest
--------------------------------------------------------------------------------

Physical cores: 8
Logical cores:  16
Memory:         15GB
Extensions:     AVX

Creating new checkpoint file: data/aa_seqs_omega_aln.fasta.ckp
----------

In [13]:
!./bin/raxml-ng --help


RAxML-NG v. 1.2.2-master released on 30.04.2024 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth, Julia Haag, Anastasis Togkousidis.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: AMD Ryzen 7 5800H with Radeon Graphics, 8 cores, 14 GB RAM

Usage: raxml-ng [OPTIONS]

Commands (mutually exclusive):
  --help                                     display help information
  --version                                  display version information
  --evaluate                                 evaluate the likelihood of a tree (with model+brlen optimization)
  --search                                   ML tree search (default: 10 parsimony + 10 random starting trees)
  --bootstrap                                bootstrapping (default: use bootstopping to auto-detect #replica

In [14]:
!./bin/raxml-ng --msa 'data/aa_seqs_omega_aln.fasta' --model VT+G4+F --prefix T1 --threads 30 --log debug 


RAxML-NG v. 1.2.2-master released on 30.04.2024 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth, Julia Haag, Anastasis Togkousidis.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: AMD Ryzen 7 5800H with Radeon Graphics, 8 cores, 14 GB RAM

RAxML-NG was called at 09-May-2024 12:03:52 as follows:

./bin/raxml-ng --msa data/aa_seqs_omega_aln.fasta --model LG+G4 --prefix T1 --threads 14 --log debug

Analysis options:
  run mode: ML tree search
  start tree(s): random (10) + parsimony (10)
  random seed: 1715231032
  tip-inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
  site repeats: ON
  logLH epsilon: general: 10.000000, brlen-triplet: 1000.000000
  fast spr radius: AUTO
  spr subtree cutoff: 1.000000
  fast CLV updates: ON
  branch lengths: proportion

In [18]:
!./bin/raxml-ng --bootstrap --msa T1.raxml.rba --model VT+G4+F --prefix T2 --threads 30 --bs-tree 300


RAxML-NG v. 1.2.2-master released on 30.04.2024 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth, Julia Haag, Anastasis Togkousidis.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: AMD Ryzen 7 5800H with Radeon Graphics, 8 cores, 14 GB RAM

RAxML-NG was called at 09-May-2024 12:32:42 as follows:

./bin/raxml-ng --bootstrap --msa T1.raxml.rba --model LG+G4 --prefix T2 --threads 14 --bs-tree 1000

Analysis options:
  run mode: Bootstrapping
  start tree(s): 
  bootstrap replicates: parsimony (1000)
  random seed: 1715232762
  tip-inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
  site repeats: ON
  logLH epsilon: general: 0.100000, brlen-triplet: 1000.000000
  fast spr radius: AUTO
  spr subtree cutoff: 1.000000
  fast CLV updates: ON
  branch lengths: 

In [16]:
!./bin/raxml-ng --support --tree T1.raxml.bestTree --bs-trees T2.raxml.bootstraps --prefix T3 --threads 30

/bin/bash: line 1: ./raxml-ng: No such file or directory


In [31]:
from io import StringIO

with open("T3.raxml.support", "r") as file:
    treedata = str(file.read())
    treedata = treedata.replace("data/genbank/", "")
    treedata = treedata.replace("_complete_genome", "")

    handle = StringIO(treedata)
    with open("best_tree.nh", "w") as ofile:
        ofile.write(treedata)


#  image generated with https://icytree.org/
![tree image](tree.png)